In [1]:
import numpy as np
import pandas as pd
import os

In [2]:
AA_serials = [item for item in os.listdir('../') if 'AA' in item]
AA_serial = 'AA-202310-002'

# Define path of an Aquagar machine
AA_machine_folder_path = f'../{AA_serial}'

# Define path of Raw Data folders
AA_RawData_folder_path = os.path.join(AA_machine_folder_path, 'Raw Data')
experiment_folders_paths = [os.path.join(AA_RawData_folder_path, item) for item in os.listdir(AA_RawData_folder_path)[1:]]

In [4]:
results_xlsx_path = os.path.join(AA_machine_folder_path, [item for item in os.listdir(AA_machine_folder_path) if 'Resultados' in  item][0])
results_df = pd.read_excel(results_xlsx_path, dtype={'Fecha': str, 'Muestra': str, 'Plate_ID':str})
results_df

,FechaExperimento,Muestra,Plate_ID,Vibrio,Aeromonas,Photobacterium,Staphyloccocus,Micrococcus,Microbiota
0,2023-11-22,Salmonetes,0006,🟢,🟢,🟢,🟢,🟢,Monopolizada
1,2023-11-22,Doradas,0006,🟢,🟢,🟢,🟢,🟢,NaN
2,2023-11-23,FilaSuperior,NaN,🟢,🟢,🟢,🔴,🟢,Monopolizada
3,2023-11-23,FilaInferior,NaN,🟢,🟢,🟢,🔴,🟢,Monopolizada
4,2023-11-24,FilaSuperior,NaN,🟢,🟢,🟢,🟢,🔴,Monopolizada
5,2023-11-24,FilaInferior,NaN,🟢,🟢,🟢,🟢,🟢,NaN
6,2023-12-20,Tanque A,0002,🟢,🟢,🟢,🟢,🟢,NaN
7,2023-12-20,Tanque B,0002,🟢,🟢,🟢,🟢,🟢,NaN
8,2023-12-27,FilaSuperior,NaN,🟡,🟢,🟢,🟢,🟢,Monopolizada
9,2023-12-27,FilaInferior,NaN,🟡,🟢,🟢,🟢,🟢,Monopolizada


In [30]:
# Define the date bounds
start_date, end_date = '2023-12-20', '2023-12-27'

In [32]:
results_df_filtered_dates = results_df[(results_df['FechaExperimento']>=start_date) & (results_df['FechaExperimento']<=end_date)]

In [94]:
df = results_df_filtered_dates

# Generate HTML code with formatting
html_code = df.to_html(
    index=False,
    formatters={col: lambda x: '' if pd.isna(x) else f'<div style="text-align:center; ">{x}</div>' for col in df.columns},
    escape=False,
    na_rep=' ',
    border=0,
    justify='center'
)

# Add additional formatting for the header row and remove borders
html_code = html_code.replace('<thead>', '<thead style="font-weight:bold; background-color:#f2f2f2; text-align:center">')
html_code = html_code.replace('<th>', '<th style="border: none">')
html_code = html_code.replace('<td>', '<td style="border: none">')


In [95]:
email_receivers = ['guillem.cobos@koabiotech.com']
name_receiver = 'Guillem'
serial_num = 'AA-202310-002'
html_tables = html_code

In [96]:
from email.message import EmailMessage
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.image import MIMEImage
import ssl
import smtplib

email_sender = 'guillem.cobos@koabiotech.com'
email_password = 'yqeq lxsp vdxl nkni'

subject = f'Resultados del Análisis Microbiológico - Informe de Laboratorio🧬🔬' 
body = f""" 
<p>Estimado/a {name_receiver},</p>
<p>Los siguientes resultados ofrecen una visión detallada de los parámetros biológicos evaluados en la muestra proporcionada.</p>
<p><strong>Detalles del Análisis:</strong><br />- Fechas del Análisis: Del {start_date} hasta el {end_date} <br />- Tipo de Muestra: Cultivo microbiológico sobre agar<br /> </p>
"""

after_report = """ 
<p><strong>Observaciones</strong>:<br />[Incluir cualquier observaci&oacute;n o nota relevante]</p>
<p><strong>Recomendaciones</strong>:<br />[En caso de ser necesario, proporcionar recomendaciones basadas en los resultados]</p>
<p>Si tiene alguna pregunta o necesita m&aacute;s informaci&oacute;n, no dude en ponerse en contacto con nuestro equipo de expertos. Agradecemos su confianza en nuestros servicios y esperamos poder atenderle en futuras ocasiones.</p>
"""

email_signature = """
<p>Atentamente,</p>
<p>Equipo de an&aacute;lisis y laboratorio.</p>
<p>&copy; 2023 KOA Biotech.</p>
<p><img src="https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTqEEqzU6gsIyaAflx43K90fwr6f04EJZtLrGz9fC8_pOY4b19ayUC0SFt0muHUsf-9jQ&amp;usqp=CAU    " alt="" width="150" height="46" /></p>
"""

em = MIMEMultipart()
em['From'] = email_sender
em['To'] = ', '.join(email_receivers)
em['Subject'] = subject

# Attach the body of the email
em.attach(MIMEText(body, 'html'))
# Attach the HTML content to the email
em.attach(MIMEText(html_tables, "html"))
em.attach(MIMEText(after_report, "html"))
em.attach(MIMEText(email_signature, "html"))

'''# Attach an image
attachment_path = img_report_path
with open(attachment_path, 'rb') as attachment:
    image = MIMEImage(attachment.read(), name='pathogen_predictions.png')
em.attach(image)'''

context = ssl.create_default_context()

with smtplib.SMTP_SSL('smtp.gmail.com', 465, context = context) as smtp:
    smtp.login(email_sender, email_password)
    smtp.sendmail(email_sender, email_receivers, em.as_string())